# Azure AI Agent Service और Semantic Kernel का उपयोग करके RAG

यह कोड स्निपेट दिखाता है कि `Azure AI Agent Service` और `Semantic Kernel` का उपयोग करके रिट्रीवल-ऑगमेंटेड जनरेशन (RAG) के लिए एक Azure AI एजेंट कैसे बनाया और प्रबंधित किया जाए। एजेंट उपयोगकर्ता के प्रश्नों को प्राप्त संदर्भ के आधार पर संसाधित करता है और तदनुसार सटीक उत्तर प्रदान करता है।


SQLite संस्करण सुधार  
यदि आपको यह त्रुटि मिले:  
```
RuntimeError: Your system has an unsupported version of sqlite3. Chroma requires sqlite3 >= 3.35.0
```  

अपने नोटबुक की शुरुआत में इस कोड ब्लॉक को अनकमेंट करें:  


In [ ]:
# %pip install pysqlite3-binary
# __import__('pysqlite3')
# import sys
# sys.modules['sqlite3'] = sys.modules.pop('pysqlite3')

### पैकेज आयात करना  
निम्नलिखित कोड आवश्यक पैकेज आयात करता है:  


In [ ]:
# Azure imports for project client and credentials
from azure.ai.projects.models import FileSearchTool, OpenAIFile, VectorStore
from azure.identity.aio import DefaultAzureCredential

# Semantic Kernel imports
from semantic_kernel.agents import AzureAIAgent, AzureAIAgentThread

# सेमांटिक कर्नल और Azure AI एजेंट सेवा के साथ रिट्रीवल-अग्मेंटेड जनरेशन

यह उदाहरण दिखाता है कि **Azure AI एजेंट सेवा** का उपयोग करके **रिट्रीवल-अग्मेंटेड जनरेशन (RAG)** कैसे किया जा सकता है, जिसमें एक भाषा मॉडल को अपलोड किए गए दस्तावेज़ से डोमेन-विशिष्ट संदर्भ के साथ जोड़ा जाता है।

### यह कैसे काम करता है

1. **दस्तावेज़ अपलोड**: एक मार्कडाउन फ़ाइल (document.md) जिसमें जानकारी (Contoso की यात्रा बीमा नीति) होती है, एजेंट सेवा में अपलोड की जाती है।

2. **वेक्टर स्टोर निर्माण**: दस्तावेज़ को वेक्टर स्टोर में इंडेक्स किया जाता है ताकि उसकी सामग्री पर सेमांटिक सर्च की जा सके।

3. **एजेंट कॉन्फ़िगरेशन**: `gpt-4o` मॉडल का उपयोग करके एक एजेंट को निम्नलिखित सख्त निर्देशों के साथ कॉन्फ़िगर किया जाता है:
   - केवल दस्तावेज़ से प्राप्त सामग्री के आधार पर प्रश्नों का उत्तर दें।
   - यदि प्रश्न दायरे से बाहर है, तो उत्तर देने से मना कर दें।

4. **फ़ाइल सर्च टूल इंटीग्रेशन**: `FileSearchTool` को एजेंट के साथ पंजीकृत किया जाता है, जिससे मॉडल को इनफेरेंस के दौरान इंडेक्स किए गए दस्तावेज़ से प्रासंगिक अंश खोजने और पुनः प्राप्त करने की अनुमति मिलती है।

5. **उपयोगकर्ता इंटरैक्शन**: उपयोगकर्ता प्रश्न पूछ सकते हैं। यदि दस्तावेज़ में प्रासंगिक जानकारी मिलती है, तो एजेंट एक आधारित उत्तर उत्पन्न करता है।  
   यदि नहीं, तो एजेंट स्पष्ट रूप से जवाब देता है कि दस्तावेज़ में पर्याप्त जानकारी उपलब्ध नहीं है।


### मुख्य फ़ंक्शन

यह मुख्य फ़ंक्शन प्रोग्राम का प्रवेश बिंदु है। यह सुनिश्चित करता है कि सभी आवश्यक घटक सही तरीके से प्रारंभ किए गए हैं।

```python
def main():
    # प्रोग्राम शुरू करने के लिए मुख्य फ़ंक्शन
    initialize_components()
    run_application()
```

### घटकों को प्रारंभ करना

यह फ़ंक्शन सभी आवश्यक घटकों को सेटअप करता है ताकि प्रोग्राम सुचारू रूप से चल सके।

```python
def initialize_components():
    # आवश्यक घटकों को प्रारंभ करें
    load_configuration()
    setup_logging()
```

### एप्लिकेशन चलाना

यह फ़ंक्शन मुख्य एप्लिकेशन को चलाने के लिए जिम्मेदार है।

```python
def run_application():
    # मुख्य एप्लिकेशन चलाएं
    while not is_terminated():
        process_events()
```

### कॉन्फ़िगरेशन लोड करना

यह फ़ंक्शन कॉन्फ़िगरेशन फ़ाइल को लोड करता है और आवश्यक सेटिंग्स लागू करता है।

```python
def load_configuration():
    # कॉन्फ़िगरेशन फ़ाइल लोड करें
    config = read_config_file("config.yaml")
    apply_settings(config)
```

### लॉगिंग सेटअप करना

यह फ़ंक्शन लॉगिंग सिस्टम को प्रारंभ करता है ताकि प्रोग्राम की गतिविधियों को ट्रैक किया जा सके।

```python
def setup_logging():
    # लॉगिंग सिस्टम सेटअप करें
    initialize_logger("application.log")
```

### घटनाओं को संसाधित करना

यह फ़ंक्शन प्रोग्राम के दौरान उत्पन्न होने वाली घटनाओं को संभालता है।

```python
def process_events():
    # घटनाओं को संसाधित करें
    event = get_next_event()
    handle_event(event)
```

### प्रोग्राम समाप्त करना

यह फ़ंक्शन प्रोग्राम को समाप्त करने की स्थिति की जांच करता है।

```python
def is_terminated():
    # प्रोग्राम समाप्त करने की स्थिति की जांच करें
    return check_termination_condition()
```


यह सुनिश्चित करें कि Azure CLI का उपयोग करके पहले `az login` चलाएं ताकि `DefaultAzureCredential` का उपयोग करते समय उचित प्रमाणीकरण संदर्भ प्रदान किया जा सके। Azure AI Agent Service API कुंजियों का उपयोग नहीं करता है।


In [ ]:
async def main():
    async with (
        DefaultAzureCredential() as creds,
        AzureAIAgent.create_client(credential=creds) as client,
    ):
        file: OpenAIFile = await client.agents.upload_file_and_poll(file_path="document.md", purpose="assistants")
        vector_store: VectorStore = await client.agents.create_vector_store_and_poll(
            file_ids=[file.id], name="my_vectorstore"
        )

        # Define agent name and instructions tailored for RAG.
        AGENT_NAME = "RAGAgent"
        AGENT_INSTRUCTIONS = """
        You are an AI assistant designed to answer user questions using only the information retrieved from the provided document(s).

        - If a user's question cannot be answered using the retrieved context, **you must clearly respond**: 
        "I'm sorry, but the uploaded document does not contain the necessary information to answer that question."
        - Do not answer from general knowledge or reasoning. Do not make assumptions or generate hypothetical explanations.
        - Do not provide definitions, tutorials, or commentary that is not explicitly grounded in the content of the uploaded file(s).
        - If a user asks a question like "What is a Neural Network?", and this is not discussed in the uploaded document, respond as instructed above.
        - For questions that do have relevant content in the document (e.g., Contoso's travel insurance coverage), respond accurately, and cite the document explicitly.

        You must behave as if you have no external knowledge beyond what is retrieved from the uploaded document.
        """

        
        # Create file search tool with uploaded resources
        file_search = FileSearchTool(vector_store_ids=[vector_store.id])

        # 3. Create an agent on the Azure AI agent service with the file search tool
        agent_definition = await client.agents.create_agent(
            model="gpt-4o",  # This model should match your Azure OpenAI deployment.
            name=AGENT_NAME,
            instructions=AGENT_INSTRUCTIONS,
            tools=file_search.definitions,
            tool_resources=file_search.resources,
        )
        
        # Create the Azure AI Agent using the client and definition.
        agent = AzureAIAgent(
            client=client,
            definition=agent_definition,
        )
        
        # Create a thread to hold the conversation
        # If no thread is provided, a new thread will be
        # created and returned with the initial response
        thread: AzureAIAgentThread | None = None
        
        # Example user queries.
        user_inputs = [
            "Can you explain Contoso's travel insurance coverage?",  # Relevant context.
            "What is a Neural Network?"  # No relevant context from the document. Will not contain a source annotation.
        ]
        
        try:
            for user_input in user_inputs:
                print(f"# User: '{user_input}'")
                # Invoke the agent for the specified thread for response
                async for response in agent.invoke(messages=user_input, thread=thread):
                    print(f"# {response.name}: {response}")
                    thread = response.thread
        finally:
            # Clean up resources.
            await thread.delete() if thread else None
            await client.agents.delete_vector_store(vector_store.id)
            await client.agents.delete_file(file.id)
            await client.agents.delete_agent(agent.id)
            print("\nCleaned up agent, thread, file, and vector store.")

await main()

आपने जो निर्देश दिए हैं, कृपया उस Markdown फ़ाइल को साझा करें जिसे आप अनुवादित करना चाहते हैं।



---

**अस्वीकरण**:  
यह दस्तावेज़ AI अनुवाद सेवा [Co-op Translator](https://github.com/Azure/co-op-translator) का उपयोग करके अनुवादित किया गया है। जबकि हम सटीकता सुनिश्चित करने का प्रयास करते हैं, कृपया ध्यान दें कि स्वचालित अनुवाद में त्रुटियां या अशुद्धियां हो सकती हैं। मूल भाषा में उपलब्ध मूल दस्तावेज़ को प्रामाणिक स्रोत माना जाना चाहिए। महत्वपूर्ण जानकारी के लिए, पेशेवर मानव अनुवाद की सिफारिश की जाती है। इस अनुवाद के उपयोग से उत्पन्न किसी भी गलतफहमी या गलत व्याख्या के लिए हम जिम्मेदार नहीं हैं।
